In [1]:
!pip install openai==0.28


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import openai
import pandas as pd
import random
import time

# ✅ 1. OpenAI(Upstage Solar) API 설정
openai.api_key = "up_en2tvM4xLkewKhm2OTJ15gbwlH8d1"  # 실제 API 키로 교체하세요.
openai.api_base = "https://api.upstage.ai/v1/solar"  # 업스테이지 Solar API 엔드포인트

# ✅ 2. 데이터 불러오기 (업로드된 CSV 파일 사용)
file_path = "train_generated.csv"
df = pd.read_csv(file_path)

# ✅ 3. 랜덤 샘플링 (각 호출마다 10개씩 예시 문장 선택)
sampled_rows = df["generated_dialogue"].dropna().tolist()

# ✅ 4. 생성할 새 문장(대화) 개수 (한 번의 API 호출당)
num_new_sentences_each_time = 10

# ✅ 5. 반복 횟수 (예: 10번 호출하여 총 100문장 생성 시도)
num_iterations = 5000

all_generated_dialogues = []

for iteration in range(num_iterations):
    # (A) 예시 문장 10개 선택
    if len(sampled_rows) >= 10:
        selected_examples = random.sample(sampled_rows, k=10)
    else:
        selected_examples = random.choices(sampled_rows, k=10)
    
    # (B) 개선된 few-shot 프롬프트 구성
    prompt = (
        "아래는 다양한 주제와 상황, 그리고 서로 다른 문체를 가진 예시 대화 문장들이야. "
        "이 예시들을 참고하여, 두 인물이 자연스럽게 대화를 주고받는 **창의적이고 풍부한** 문장을 만들어줘.\n\n"
        "조건:\n"
        "1) 예시 문장의 형식을 그대로 복사하지 말고, 새로운 내용을 포함할 것.\n"
        "2) 문장은 최소 40단어 이상으로 충분한 분량을 가지며, 일상적인 상황, 감정, 분위기를 생생하게 묘사할 것.\n"
        "3) 대화 형식은 한 문단 내에서 자연스럽게 전개되도록 하며, 화자나 인물의 이름, 직책 등은 언급하지 않을 것.\n"
        "4) 문장은 반드시 '1.'로 시작하고, 번호는 1부터 순차적으로 증가시킬 것.\n"
        "5) 폭력적, 혐오적, 과도하게 선정적인 표현은 사용하지 않을 것.\n"
        f"6) 총 {num_new_sentences_each_time}개의 문장을 생성해줘.\n\n"
        "아래는 예시 문장 목록이야:\n\n"
    )
    
    for i, ex in enumerate(selected_examples, start=1):
        prompt += f"예시 {i}: {ex}\n"
    
    prompt += (
        "\n\n위 예시들을 참고하여, 서로 다른 상황과 감정을 담은 두 인물 간의 대화를 한 문단 내에서 "
        "자연스럽고 상세하게 표현해줘. 창의적인 전개와 다양한 어휘를 사용하여 풍부한 문장을 만들어줘."
    )
    
    # (C) Upstage Solar API 호출 (스트리밍 모드)
    stream = openai.ChatCompletion.create(
        model="solar-pro",
        messages=[{"role": "user", "content": prompt}],
        stream=True,
        temperature=1,
        max_tokens=30000
    )
    
    # (D) 스트리밍 응답 수집
    generated_text = ""
    for chunk in stream:
        if "choices" in chunk and len(chunk.choices) > 0:
            delta = chunk.choices[0].delta
            if "content" in delta:
                generated_text += delta["content"]
    
    # (E) 숫자로 시작하는 줄(“1. …”)만 추출
    dialogues = [
        line.strip()
        for line in generated_text.strip().split("\n")
        if line.strip() and line.strip()[0].isdigit()
    ]
    
    all_generated_dialogues.extend(dialogues)
    print(f"[{iteration+1}/{num_iterations}] 이번에 생성된 대화 문장 수: {len(dialogues)} | 누적 대화 문장 수: {len(all_generated_dialogues)}")
    
    # (F) API 과부하 방지를 위한 대기 (필요 시 조정)
    time.sleep(2)

# ✅ 6. 최종 결과 개수 확인
print(f"\n🎉 총 생성된 대화 문장 수: {len(all_generated_dialogues)}")

# ✅ 7. CSV 파일로 저장
df_generated = pd.DataFrame(all_generated_dialogues, columns=["generated_dialogue"])
df_generated.to_csv("daily_conversation_from_train.csv", index=False)

print("\n✅ 두 사람 대화 형식 문장 생성 완료! CSV 파일로 저장되었습니다.")
print("👉 파일명: daily_conversation_from_train.csv")



[1/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 10
[2/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 20
[3/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 30
[4/5000] 이번에 생성된 대화 문장 수: 5 | 누적 대화 문장 수: 35
[5/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 45
[6/5000] 이번에 생성된 대화 문장 수: 1 | 누적 대화 문장 수: 46
[7/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 56
[8/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 66
[9/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 76
[10/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 86
[11/5000] 이번에 생성된 대화 문장 수: 5 | 누적 대화 문장 수: 91
[12/5000] 이번에 생성된 대화 문장 수: 5 | 누적 대화 문장 수: 96
[13/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 106
[14/5000] 이번에 생성된 대화 문장 수: 3 | 누적 대화 문장 수: 109
[15/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 119
[16/5000] 이번에 생성된 대화 문장 수: 1 | 누적 대화 문장 수: 120
[17/5000] 이번에 생성된 대화 문장 수: 2 | 누적 대화 문장 수: 122
[18/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 132
[19/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 142
[20/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 152
[21/5000] 이번에 생성된 대화 문장 수: 10 | 누적 대화 문장 수: 162
[22/5000] 이번에 생성된 대화

KeyboardInterrupt: 

In [28]:
df_generated = pd.DataFrame(all_generated_dialogues, columns=["generated_dialogue"])
df_generated.to_csv("daily_conversation_from_train.csv", index=False)